# 1. Introduction to Tecton on Snowflake

## 1) Setup

Before getting started, lets do some setup to get your computer ready to interact with Tecton

### 1.1) Install the Tecton CLI on your local machine

<div class="alert alert-block alert-warning">
Tecton requires Python version 3.7, 3.8, or 3.9 to run. We also recommend installing tecton into a Python virtual environment.
</div>

To install the Tecton CLI on your local machine, run:

✅ `$ pip install --pre tecton`

If you run into any issues, follow [these instructions in the Tecton Docs](https://docs.tecton.ai/v2/setting-up-tecton/02-tecton-cli-setup.html) to set up the Tecton CLI.

Once you have finished installing the CLI, you can log in to your Tecton cluster using:

✅ `$ tecton login my-cluster.tecton.ai`

You will also need to configure access to your Snowflake account using:

✅ `$ tecton snowflake configure`

### 1.2) Clone the Tecton Sample Repository

This tutorial will use [a sample repository full of pre-built features and data sources](https://github.com/tecton-ai-ext/tecton-snowflake-feature-repo).

Before you get started, clone this repository to your local machine using:

✅ `$ git clone https://github.com/tecton-ai-ext/tecton-snowflake-feature-repo.git`

### 1.3) Install the Snowflake Connector for Python

This tutorial makes use of the Snowflake Python Connector to execute queries against Snowflake and return the results as a Pandas DataFrame.

To install the Snowflake Python Connector on your local machine, run:

✅ `$ pip install 'snowflake-connector-python[pandas]'`

### 1.4) Import some packages and check that Tecton is installed

✅ Run the cell below, but make sure to replace the Snowflake connection parameters with your account info.

In [1]:
# Import Tecton and other libraries
import tecton
import pandas as pd
import snowflake.connector
from dotenv import dotenv_values
from datetime import datetime, timedelta
from pprint import pprint

env = {**dotenv_values(".env")}

# Quick helper function to query snowflake from a notebook
# Make sure to replace with the appropriate connection details for your own account
def query_snowflake(query):
    conn = snowflake.connector.connect(
        user=env['SNOWFLAKE_USER'],
        password=env['SNOWFLAKE_PWD'],
        account=env['SNOWFLAKE_ACCOUNT'],
        warehouse=env['SNOWFLAKE_WAREHOUSE']
    )
    df = conn.cursor().execute(query).fetch_pandas_all()
    return df

tecton.version.summary()

Version: 0.4.0b16
Git Commit: 7627b2f1c1a965a458440f3ce900a907671f2185
Build Datetime: 2022-04-05T23:41:41


## 2) Interacting with Tecton
Your Tecton account has been seeded with data and some example features that you can use to test out Tecton.

First, you can check out some of the raw data that has been connected to Tecton -- historical transactions.  You'll notice we first select the [Tecton workspace](https://docs.tecton.ai/overviews/workspaces.html) that contains the objects we want to fetch.


In [2]:
# Check out the data source in Snowflake
ws = tecton.get_workspace('prod')
ds = ws.get_data_source('transactions')
ds.summary()

,
Name,transactions
Workspace,prod
Description,
Created At,2022-04-06 23:09:11 UTC
Owner,
Last Modified By,matt@tecton.ai
Family,
Source Filename,data_sources/transactions.py
Tags,{}
Batch Data Source,Type Snowflake URL Database TECTON_DEMO_DATA Schema FRAUD_DEMO


### 2.1) Preview the raw data directly

In [3]:
# Preview the data directly
transactions_query = '''
SELECT 
    *
FROM 
    TECTON_DEMO_DATA.FRAUD_DEMO.TRANSACTIONS 
ORDER BY TIMESTAMP DESC
LIMIT 50
'''
transactions = query_snowflake(transactions_query)
transactions.head(5)

INFO - 04/06/2022 04:12:50 PM - snowflake.connector.connection - Snowflake Connector for Python Version: 2.7.6, Python Version: 3.8.13, Platform: macOS-12.2.1-x86_64-i386-64bit
INFO - 04/06/2022 04:12:50 PM - snowflake.connector.connection - This connection is in OCSP Fail Open Mode. TLS Certificates would be checked for validity and revocation status. Any other Certificate Revocation related exceptions or OCSP Responder failures would be disregarded in favor of connectivity.
INFO - 04/06/2022 04:12:50 PM - snowflake.connector.connection - Setting use_openssl_only mode to False
INFO - 04/06/2022 04:12:51 PM - snowflake.connector.cursor - query: [SELECT * FROM TECTON_DEMO_DATA.FRAUD_DEMO.TRANSACTIONS ORDER BY TIMESTAMP DESC L...]
INFO - 04/06/2022 04:12:52 PM - snowflake.connector.cursor - query execution done


,USER_ID,TRANSACTION_ID,CATEGORY,AMT,IS_FRAUD,MERCHANT,MERCH_LAT,MERCH_LONG,TIMESTAMP
0,user_461615966685,d8b3202fff63d822d2d571ec143bf4b2,grocery_net,47.85,0,fraud_Gutmann Ltd,36.128657,-87.871920,2022-03-28 21:24:25.911598
1,user_91355675520,8de05ad4920414a18188579f0e95db3a,home,33.94,0,fraud_Schmidt-Larkin,36.452992,-77.180697,2022-03-28 21:24:23.642953
2,user_394495759023,e2836492645bb49c96c59d854878b954,kids_pets,18.10,0,fraud_Rau-Grant,29.909827,-82.201760,2022-03-28 21:24:21.283878
3,user_469998441571,544f48d93d5497e874e0a24a6685c062,grocery_pos,164.26,0,"fraud_Heller, Gutmann and Zieme",39.291001,-95.947384,2022-03-28 21:24:18.348427
4,user_699668125818,31c17ad111202bb18db2fb73461c00f7,entertainment,1.82,0,fraud_Grimes LLC,39.685288,-119.278198,2022-03-28 21:24:16.493649


### 2.2) Tecton Feature Views

In Tecton, features are registered as [Feature Views](https://docs.tecton.ai/overviews/framework/feature_views/feature_views.html).  These views contain all of the information needed to transform raw data (like transactions) into features (like what % of historical transactions at a given merchant ended up being fraudulant)?

Feature Views make can make feature data available in two places:
* Offline: You can retreive historical feature values using [time travel](https://www.tecton.ai/blog/time-travel-in-ml/)
* Online: You can retreive current feature values in real time via Tecton's [real-time serving API](https://docs.tecton.ai/examples/fetch-real-time-features.html)

Feature Views can also be ran ad-hoc for testing or previewing data using `.run()`. Let's run the "Merchant Fraud Rate" Feature View to view feature data from the last 30 days (sorted by the merchants with the highest fraud rate):

In [4]:
fv = ws.get_feature_view('merchant_fraud_rate')

start_time = datetime.utcnow()-timedelta(days=30)
end_time = datetime.utcnow()

features = fv.run(feature_start_time=start_time, feature_end_time=end_time).to_pandas()

features.sort_values(by="IS_FRAUD_MEAN_72H_1D", ascending=False).head(5)

INFO - 04/06/2022 04:12:52 PM - snowflake.connector.connection - closed
INFO - 04/06/2022 04:12:52 PM - snowflake.connector.connection - No async queries seem to be running, deleting session


,MERCHANT,TIMESTAMP,IS_FRAUD_MEAN_24H_1D,IS_FRAUD_MEAN_72H_1D,IS_FRAUD_MEAN_168H_1D,IS_FRAUD_MEAN_960H_1D
1272,fraud_Kessler Group,1648339200000000000,0.090909000000,0.090909000000,0.090909000000,0.090909000000
696,"fraud_Stehr, Jewess and Schimmel",1648425600000000000,0.086957000000,0.086957000000,0.086957000000,0.086957000000
2246,fraud_Schamberger-O'Keefe,1648339200000000000,0.071429000000,0.071429000000,0.071429000000,0.071429000000
296,fraud_Funk Group,1648425600000000000,0.066667000000,0.066667000000,0.066667000000,0.066667000000
161,"fraud_Zboncak, Rowe and Murazik",1648339200000000000,0.066667000000,0.066667000000,0.066667000000,0.066667000000


## 3) Generating Training Data
Once you've built a number of features, you'll want to join them together to generate training data. 

### 3.1) Tecton Feature Services
In Tecton, features that are needed for training or predictions are grouped together into a [Feature Service](https://docs.tecton.ai/overviews/framework/feature_services.html). Let's check out a Feature Service that we've already built.

In [5]:
fs = ws.get_feature_service('fraud_detection_feature_service')
fs.summary()

Property,Value
Name,fraud_detection_feature_service
Owner,
Last Updated By,matt@tecton.ai
Description,
Family,
Entities,"['fraud_user', 'category']"
Online Serving,Enabled
Logging,Disabled
Online Join Keys,"USER_ID, CATEGORY"
Offline Join Keys,"USER_ID, CATEGORY"


The `fraud_detection_feature_service` is comprised of 13 features that are meant to be used together to train a fraud detection model.

### 3.2) Building a Spine

Let's use the `fraud_detection_feature_service` to train a model that scores transactions as either "Fraudulent" or "Non-Fraudulent".  To start, lets look up some labeled transactions that we'll use for training.

We can see in the summary above that the `fraud_detection_feature_service` requires `MERCHANT`, `USER_ID`, and `CATEGORY` join keys in order to fetch all the relevant features. Together with an event timestamp and label column, this represents our list of historical training events. In Tecton we call this a "spine".

> 💡 A spine is expected to include the entity join keys for the Feature Views in a Feature Service as well as a timestamp column for time-travel lookups. A label column is not strictly necessary but is typically included as a best practice.

In [6]:
# Preview the data directly
transactions_query = '''
SELECT 
    MERCHANT,
    USER_ID,
    CATEGORY,
    TIMESTAMP,
    IS_FRAUD
FROM 
    TECTON_DEMO_DATA.FRAUD_DEMO.TRANSACTIONS 
ORDER BY TIMESTAMP DESC
LIMIT 1000
'''
transactions = query_snowflake(transactions_query)
transactions.head(5)

,MERCHANT,USER_ID,CATEGORY,TIMESTAMP,IS_FRAUD
0,fraud_Gutmann Ltd,user_461615966685,grocery_net,2022-03-28 21:24:25.911598,0
1,fraud_Schmidt-Larkin,user_91355675520,home,2022-03-28 21:24:23.642953,0
2,fraud_Rau-Grant,user_394495759023,kids_pets,2022-03-28 21:24:21.283878,0
3,"fraud_Heller, Gutmann and Zieme",user_469998441571,grocery_pos,2022-03-28 21:24:18.348427,0
4,fraud_Grimes LLC,user_699668125818,entertainment,2022-03-28 21:24:16.493649,0


### 3.3) Getting Training Data with `get_historical_features`

To retreive training data, we'll use Tecton's `get_historical_features` API, which allows us to join the 13 features contained in `fraud_detection_feature_service` onto our historical transactions.


A Feature Service will expect a spine in the form of a Pandas Dataframe or a Snowflake query that generates the events as shown below.

In [7]:
training_data = fs.get_historical_features(spine=transactions_query, timestamp_key="TIMESTAMP").to_pandas()
training_data.head(10)

,USER_ID,CATEGORY,TIMESTAMP,MERCHANT,IS_FRAUD,USER_TRANSACTION_METRICS__TRANSACTION_SUM_24H_1D,USER_TRANSACTION_METRICS__TRANSACTION_SUM_72H_1D,USER_TRANSACTION_METRICS__TRANSACTION_SUM_168H_1D,USER_TRANSACTION_METRICS__TRANSACTION_SUM_960H_1D,USER_TRANSACTION_METRICS__AMT_MEAN_24H_1D,USER_TRANSACTION_METRICS__AMT_MEAN_72H_1D,USER_TRANSACTION_METRICS__AMT_MEAN_168H_1D,USER_TRANSACTION_METRICS__AMT_MEAN_960H_1D,USER_CATEGORY_COUNT__TRANSACTION_SUM_24H_1D,USER_CATEGORY_COUNT__TRANSACTION_SUM_72H_1D,USER_CATEGORY_COUNT__TRANSACTION_SUM_168H_1D,USER_CATEGORY_COUNT__TRANSACTION_SUM_960H_1D
0,user_461615966685,grocery_net,2022-03-28 21:24:25.911598,fraud_Gutmann Ltd,0,758.0,1698.0,1698.0,1698.0,76.155976,79.900878,79.900878,79.900878,21.0,51.0,51.0,51.0
1,user_91355675520,home,2022-03-28 21:24:23.642953,fraud_Schmidt-Larkin,0,170.0,387.0,387.0,387.0,85.774000,85.438088,85.438088,85.438088,11.0,24.0,24.0,24.0
2,user_394495759023,kids_pets,2022-03-28 21:24:21.283878,fraud_Rau-Grant,0,998.0,2347.0,2347.0,2347.0,62.107946,61.398969,61.398969,61.398969,76.0,200.0,200.0,200.0
3,user_469998441571,grocery_pos,2022-03-28 21:24:18.348427,"fraud_Heller, Gutmann and Zieme",0,718.0,1609.0,1609.0,1609.0,66.713384,65.631889,65.631889,65.631889,67.0,146.0,146.0,146.0
4,user_699668125818,entertainment,2022-03-28 21:24:16.493649,fraud_Grimes LLC,0,701.0,1599.0,1599.0,1599.0,73.667846,67.225660,67.225660,67.225660,41.0,117.0,117.0,117.0
5,user_459842889956,home,2022-03-28 21:24:13.051564,fraud_Schmidt-Larkin,0,713.0,1610.0,1610.0,1610.0,59.461992,57.531087,57.531087,57.531087,69.0,140.0,140.0,140.0
6,user_205125746682,personal_care,2022-03-28 21:24:10.868077,fraud_Kub PLC,0,510.0,1217.0,1217.0,1217.0,57.796451,59.354782,59.354782,59.354782,41.0,78.0,78.0,78.0
7,user_461615966685,shopping_pos,2022-03-28 21:24:08.516627,fraud_Pouros-Conroy,0,758.0,1698.0,1698.0,1698.0,76.155976,79.900878,79.900878,79.900878,67.0,154.0,154.0,154.0
8,user_222506789984,shopping_pos,2022-03-28 21:24:04.821814,fraud_Pacocha-Bauch,0,2034.0,4614.0,4614.0,4614.0,70.685801,68.356194,68.356194,68.356194,221.0,479.0,479.0,479.0
9,user_205125746682,health_fitness,2022-03-28 21:24:02.137302,fraud_Schroeder Group,0,510.0,1217.0,1217.0,1217.0,57.796451,59.354782,59.354782,59.354782,28.0,84.0,84.0,84.0


### What is happening behind the scenes

Behind the scenes, Tecton is doing a row-level, [point-in-time correct](https://www.tecton.ai/blog/time-travel-in-ml/) join.  This join logic helps you ensure that the data you use to train your models is drawn from the same distribution as the data that is likely to be used at production time.

One other helpful thing -- you never need to worry about different concepts of time in your data when generating training data. For each feature you can specify the most convenient or correct time for that feature, and Tecton's join logic will make it easy to join all of your features together.

<img src="https://docs.tecton.ai/v2/assets/docs/examples/point-in-time-correct-joins.png" width="50%" />

## 4) Getting Real-Time Features for Inference

### 4.1) Generate an API token

To fetch real-time (online) features at low latency for a production application we will use Tecton's REST API.

This will require creating an API key. In your terminal, run:

✅ `$ tecton api-key create`

Then set this API key as an enviornment variable using the line below and replacing "<key>" with the generate API key:

✅ `$ export TECTON_API_KEY=<key>`

### 4.2) Retrieve online features using the Python SDK

We can hit Tecton's REST API dirctly from the Python SDK using `fs.get_online_features(keys)`. This is convenient for testing purposes.

✅ To query the REST API from the Python SDK, we need to set the API key in the first line of the cell below. Replace "\<key>" with the token generated in the step above.

In [11]:
tecton.conf.set("TECTON_API_KEY", "<key>")

keys = {
    # 'MERCHANT': 'fraud_Gutmann Ltd',
    'USER_ID': 'user_461615966685',
    'CATEGORY': 'grocery_net'
}
features = fs.get_online_features(join_keys=keys).to_dict()
pprint(features)

{'user_category_count.TRANSACTION_SUM_168H_1D': None,
 'user_category_count.TRANSACTION_SUM_24H_1D': None,
 'user_category_count.TRANSACTION_SUM_72H_1D': None,
 'user_category_count.TRANSACTION_SUM_960H_1D': 65,
 'user_transaction_metrics.AMT_MEAN_168H_1D': None,
 'user_transaction_metrics.AMT_MEAN_24H_1D': None,
 'user_transaction_metrics.AMT_MEAN_72H_1D': None,
 'user_transaction_metrics.AMT_MEAN_960H_1D': 81.16982954545455,
 'user_transaction_metrics.TRANSACTION_SUM_168H_1D': None,
 'user_transaction_metrics.TRANSACTION_SUM_24H_1D': None,
 'user_transaction_metrics.TRANSACTION_SUM_72H_1D': None,
 'user_transaction_metrics.TRANSACTION_SUM_960H_1D': 2288}


### 4.3) Retrieve features directly from the REST API via a cURL

We can also directly query Tecton's REST API using the example cURL below.

✅ Run this in your terminal, but make sure to replace the "my-cluster" cluster name in the first line with your cluster name:

```bash
curl -X POST --silent https://my-cluster.tecton.ai/api/v1/feature-service/get-features\
     -H "Authorization: Tecton-key $TECTON_API_KEY" -d\
'{
  "params": {
    "feature_service_name": "fraud_detection_feature_service",
    "join_key_map": {
      "USER_ID": "user_461615966685",
      "CATEGORY": "grocery_net"
    },
    "workspace_name": "prod"
  }
}' | jq
```

# What's Next

Tecton is a powerful tool to build, manage, share, and consume features for ML.  Check out the next tutorial "Creating Features on Snowflake" to learn how to build your own features.